# Ditch Dirty Data

Welcome to your SQL Data Cleaning lab! In this lab, we are going to work with a dirty dataset from Monster.com that lists job postings from February 2016 to January 2017. We need a clean dataset so that we can look at manager job trends across the USA for a data visualization project. 

For this lab, we’re just preparing the dataset and not making any visualizations, but it’s important to know why you’re cleaning and how you’ll use the data before you start cleaning up.


## Welcome to SQL!

SQL stands for Structured Query Language. It's a programming language that lets us store and retrieve important business data stored in _relational databases_. In the most basic terms, you can think of a relational database as a set of spreadsheets linked together. In SQL, we call these spreadsheets **tables**.

### Table Structure

If you've worked with spreadsheets before, SQL tables will look quite familiar. Tables are comprised rows and columns. Each row is called a **record**, and each column represents an **attribute** of that record.

In our example dataset, each row represents a newly posted job listing on monster.com. The columns simply tell us more about that job by describing its attributes like `job_title`, `location`, or `salary`:

**TODO: INSERT AN ILLUSTRATION**


### Connecting to our Database

The first step in working with SQL is connecting to our database. To get warmed up, try running the following command to create a new connection to our `jobs` database.

_Note: You can use `SHIFT + RETURN` on any code block in this lab to run the code_


In [ ]:
## Load SQL
%load_ext sql

## Connect to our SQL database called "jobs"
%sql sqlite:///jobs.db

## Querying Our Database

The first step in cleaning our dataset is being able to find the data we care about. SQL lets us write **queries** to do this.

SQL queries generally start with a `SELECT` statement. `SELECT` helps us find the specific rows, or records, that match our query criteria.

Let’s start with a simple query to get a taste of what our dataset looks like:

In [ ]:
%%sql

SELECT * FROM jobs LIMIT 10;


So what's happening here? `SELECT` statements generally follow the format:

```sql
SELECT [columns] FROM [table_name] ... [QUERY FILTERS]
```

Here, we used the `*` operator as a shortcut for "all columns", and `LIMIT 2` as a query filter to only return the first 2 matching rows in our database.

#### WHERE

Instead of limiting ourselves to the first 2 results, let's see if there's other ways to filter the data. We can use the `WHERE` statement to help by giving our query specific criteria:

A common pattern for filtering is:

```sql
WHERE [column_name] = [value]
```

Let's try it here by looking for the first 10 jobs that haven't expired:


In [ ]:
%%sql
SELECT * FROM jobs WHERE has_expired = 'No' LIMIT 10;

We can use the `AND` operator to make more specific conditions for the `WHERE` statement.

Let's update our query to look for active jobs in the U.S.:

In [ ]:
%%sql

SELECT * FROM jobs WHERE has_expired = 'No' AND country_code = 'US' LIMIT 10;

Simple queries are ok on one line. But the bigger they get, it’s easier to space them out for readability. Let’s make this query a little easier to read by spacing it on multiple lines.

In the notebook, type:


SELECT * 
FROM jobs 
WHERE has_expired = ‘n’ 
AND country_code = ‘US’;

That query isn’t really filtering any better data. We have some columns like “country” and “has expired” that don’t add interesting information or distinctions in the data. Let’s clean it up and extract the data we care about to start to explore the trends. By adding only the columns we are interested in to the select clause, the query output becomes more useful. 

In the notebook, type:

SELECT job_type, job_title, location
FROM jobs 
WHERE has_expired = ‘n’ 
AND country_code = ‘US’

This is much easier to read, but there are still too many jobs to find meaningful trends. Let’s filter and try to classify the dataset and look at just the manager jobs in the dataset to see how manager jobs differ by state. By adding another AND clause we can search for job titles that are “Manager”.

In the notebook, type:

SELECT job_type, job_title, location
FROM jobs  
WHERE has_expired = ‘n’ 
AND country_code = ‘US’
AND job_title = ‘Manager’;

That was too restrictive of a search. Let’s use a wildcard to make sure we find all the instances. Wildcards are special characters that can stand in for unknown characters in a text value. For example our dataset may have included “product manager”. The wild card will make sure we don’t miss this entry. In SQL you use a % to enter a wildcard. 

In the notebook, type:

SELECT job_type, job_title, location
FROM jobs 
WHERE has_expired = ‘n’ 
AND country_code = ‘US’
AND job_title like ‘%Manager%’;

Now we have all the manager jobs in the USA, but what about  “management” jobs? What if the dataset characterized manager roles with similar, but different terms? We won’t want to miss those in our analysis. You could try using different terms in your query to see the different results:

In the notebook, type:

SELECT job_type, job_title, location
FROM jobs 
WHERE has_expired = ‘n’ 
AND country_code = ‘US’
AND job_title like ‘%Manage%’;

Or you could check for both by using an OR clause. Be sure to add parentheses around your OR clause to keep it a logical unit. Which version do you like better? The single view, or the combination? Which is easier to read?

In the notebook, type:

SELECT job_type, job_title, location
FROM jobs 
WHERE has_expired = ‘n’ 
AND country_code = ‘US’
AND (job_title like ‘%Manager%’ OR job_title like ‘%Management%’);

What if you try a different capitalization of letters in your query? Does that change your results?

In the notebook, type:

SELECT job_type, job_title, location
FROM jobs 
WHERE has_expired = ‘n’ 
AND country_code = ‘US’
AND job_title like ‘%manage%’;

Checking for all the capitalization combinations can be tedious. We can use the “upper” function to capitalize all often characters in a column for easy comparison.

In the notebook, type:

SELECT job_type, job_title, location
FROM jobs 
WHERE has_expired = ‘n’ 
AND country_code = ‘US’
AND upper(job_title) like ‘%MANAG%’;

I wonder if we’re missing anything else? To ensure we’re working with complete data, we need to know if we really have all the manager jobs. Let’s check the job description by using the same syntax that we used on the job title.

In the notebook, type:

SELECT job_type, job_title, location
FROM jobs 
WHERE has_expired = ‘n’ 
AND country_code = ‘US’
AND upper(job_description) like ‘%MANAG%’;

Looks like there are even more manager jobs! So now let’s try searching both columns in the same query.

In the notebook, type:

SELECT job_type, job_title, location
FROM jobs 
WHERE has_expired = ‘n’ 
AND country_code = ‘US’
AND (upper(job_description) like ‘%MANAG%’
OR upper(job_title) like ‘%MANAG%’);

Are you starting to feel confident you found all your manager jobs? Great!

Our data could still be cleaner! We can add a column called management which aggregates all of this management classification.

In the notebook, type:

ALTER TABLE  jobs ADD management text;

Now you have one trustworthy management column for all the sources! Let’s add the data to it with an update statement.

In the notebook, type:

UPDATE TABLE  jobs SET management = ‘y’
WHERE (upper(job_description) like ‘%MANAG%’
OR upper(job_title) like ‘%MANAG%’);

Now we have to classify the non-management jobs in your new column with the same update statement but reversing the logic with a NOT keyword.

In the notebook, type:

UPDATE TABLE  jobs SET management = ‘n’
WHERE NOT (upper(job_description) like ‘%MANAG%’
OR upper(job_title) like ‘%MANAG%’);

Try out your new column in a query.

In the notebook, type:

SELECT job_type, job_title, location
FROM jobs 
WHERE has_expired = ‘n’ 
AND country_code = ‘US’
AND management = ‘y’;
Dig Deeper

Let’s try these techniques again. Instead of focusing on manager as a classification, choose another job type - sales, engineering, etc.  Repeat the steps above and classify more data!








Ditch Dirty Data: Consistent Data

This is great, you have made your dataset cleaner and easier to use by classifying the job types! Let’s make it even easier to use by making our data more consistent. You are looking at trends for full time management roles in the USA. Let’s start by investigating the job type column.

In the notebook, type:

SELECT job_type, job_title, location
FROM jobs 
WHERE job_type = ‘Full Time’;

The query returns lots of jobs, but are we sure we have all the full time jobs? Let’s widen the search again and consider other values. Let’s include a wild card again to make sure we don’t miss any entries.

In the notebook, type:

SELECT job_type, job_title, location
FROM jobs 
WHERE job_type like ‘Full Time%’;

Capitalizations can be a challenge as well. We can add the function upper before the column name to capitalize all the entries. This will ensure we don’t miss any because of different capitalizations. 

In the notebook, type:

SELECT job_type, job_title, location
FROM jobs 
WHERE upper(job_type) = ‘%FULL TIME%’;

How many versions of full time can you find? Let’s create a summary by making a count and grouping by the job type. Adding the function count(*) will count the entries, and adding GROUP BY will group the count by job type. 

In the notebook, type:

SELECT count(*), job_type
FROM jobs 
GROUP BY job_type;

That’s a lot of data. Let’s sort it by adding an ORDER BY COUNT that will sort the job types by the number of occurrences.

In the notebook, type:

SELECT count(*), job_type
FROM jobs 
GROUP BY job_type
ORDER BY COUNT(*);

Alternatively, you can reverse the sort by adding a desc for descending.

In the notebook, type:

SELECT count(*), job_type
FROM jobs 
GROUP BY job_type
ORDER BY COUNT(*) desc;

There are a lot of variants of full time employees, let’s standardize them by modifying the values in the table.

In the notebook, type:

UPDATE TABLE  jobs SET job_type = ‘Full Time’
WHERE job_type = ‘Full Time Employee’;

Good start, but did you get them all? Check the count query again.

In the notebook, type:

SELECT count(*), job_type
FROM jobs 
GROUP BY job_type
ORDER BY COUNT(*);

You can add wildcards in the middle of a search too to find even more relevant entries.

In the notebook, type:
UPDATE TABLE  jobs SET job_type = ‘Full Time’
WHERE job_type = ‘Full Time%Employee’;

How many did you get now? See if there are any other entries that you want to clean up. 

In the notebook, type:

SELECT count(*), job_type
FROM jobs 
GROUP BY job_type
ORDER BY COUNT(*);

You know what is annoying in data? No data. How many of those job titles are blank? By using the “is null” syntax in your query you can find the entries that are empty. 

In the notebook, type:

SELECT job_title, job_description
FROM jobs 
WHERE job_type is null;

After scanning those jobs, let’s assume they are full time.  Update them in the table to full time.

In the notebook, type:

UPDATE TABLE  jobs SET job_type = ‘Full Time’
WHERE job_type is null;
Dig Deeper

How consistent does this data seem now? Do you trust the job types enough? Do some more clean up if you think you can find more issues! Look for entries that seem completely wrong and repeat the steps above to improve your dataset.


Ditch Dirty Data: Splitting Data

Next, let’s investigate the location of these jobs by counting the different locations in the dataset. There’s just one column to describe the location, start by counting up the unique entries.

In the notebook, type:

SELECT count(*), location
FROM jobs 
GROUP BY location
ORDER BY COUNT(*) desc;

Wow, lots of issues here! The city, state and zipcode are all combined in one field. Let’s get started by adding some new columns.

In the notebook, type:

ALTER TABLE  jobs ADD city text;


ALTER TABLE  jobs ADD state text;


ALTER TABLE  jobs ADD zip text;

We need to start by finding some zip codes. To start with, use the “right” function to get the rightmost five characters from the location. Let’s assume the last 5 characters are always the zip code.

In the notebook, type:

SELECT count(*),  right(location, 5)
FROM jobs 
GROUP BY right(location, 5)
ORDER BY COUNT(*) desc;

Well that has too much noise, not all fields have zip codes and not all fields even look like valid locations. Let’s use a new zip codes reference table to check if our last 5 characters are actually a valid zip code. We do this with the IN clause and embedding a new select statement.

In the notebook, type:

SELECT count(*),  right(location, 5)
FROM jobs 
WHERE right(location, 5) IN (SELECT zip from zipcodes)
GROUP BY right(location, 5)
ORDER BY COUNT(*) desc;

Now that we have validated the data in the zipcodes table we can use the IN clause in an UPDATE statement to update our new zip code column.

In the notebook, type:

UPDATE jobs SET zip = right(location,5)
WHERE right(location, 5) IN (SELECT zip from zipcodes)
GROUP BY right(location, 5)
ORDER BY COUNT(*) desc;

How about the States next? Scan the data for patterns.

In the notebook, type:

SELECT count(*), location
FROM jobs 
GROUP BY location
ORDER BY COUNT(*) desc;

Let’s check out the data for state and no zip codes (for example “Dallas, TX”).

In the notebook, type:

SELECT count(*),  right(location, 2)
FROM jobs 
WHERE right(location, 2) IN (SELECT zip from zipcodes)
GROUP BY right(location, 2)
ORDER BY COUNT(*) desc;

Do you trust that data? How about when you check for 3 characters, you’d expect that extra character to be a space first before the State ID and how adding an “upper” would have been an issue?

In the notebook, type:

SELECT count(*),  right(location, 3)
FROM jobs 
GROUP BY right(location, 3)
ORDER BY COUNT(*) desc;

Hint: check out: 

In the notebook, type:

SELECT count(*),  right(location, 3), upper(right(location,2))
FROM jobs 
WHERE location = ‘Big Data Architect’;

Can we try another way to get the state? Is it better to assume it is the 2 characters after a comma? For example: Orlando, FL 32816. Since city names are variable length we can retrieve the first 3 characters after a comma. Use the “instr” function to find the position of the comma and the substring to get the next 3 characters.

In the notebook, type:

SELECT count(*),  substring(location, instr(location, ‘,’),3)
FROM jobs 
GROUP BY substring(location, instr(location, ‘,’),3)
ORDER BY COUNT(*) desc;

Adding the validation check back into the query for the state and using the “right” function to get just the two characters.

In the notebook, type:

SELECT count(*),  right(substring(location, instr(location, ‘,’),3),2)
FROM jobs 
WHERE right(substring(location, instr(location, ‘,’),3), 2) 
IN (SELECT state_id from zipcodes)
GROUP BY right(substring(location, instr(location, ‘,’),3),2)
ORDER BY COUNT(*) desc;

Now you have a consistent way to get a valid state, you can update the your new state field.

In the notebook, type:

UPDATE jobs SET state =  right(substring(location, instr(location, ‘,’),3),2)
WHERE right(substring(location, instr(location, ‘,’),3), 2) 
IN (SELECT state_id from zipcodes);

By modifying the use of instr and substring, we can use similar logic to retrieve the city. But now we assume the city is everything from the first character to the comma. To exclude the comma we want the character position one less than its location.

In the notebook, type:

SELECT count(*),  substring(location, 1,instr(location, ‘,’)-1)
FROM jobs 
WHERE substring(location, 1, instr(location, ‘,’)-1)
IN (SELECT state_id from zipcodes)
GROUP BY substring(location, 1, instr(location, ‘,’)-1)
ORDER BY COUNT(*) desc;

By modifying the use of instr and substring, we can use similar logic to retrieve the city.

In the notebook, type:

UPDATE jobs set city = substring(location, 1,instr(location, ‘,’)-1)
WHERE substring(location, 1, instr(location, ‘,’)-1)
IN (SELECT city from zipcodes);
Dig Deeper

Our solution to parsing out cities and states made assumptions around the format of the location data. Write a query to review how the new city and state data was populated and correct any major issues with an update statement that targets just one row at a time. 

Ditch Dirty Data: Complete Your Analysis

We now have a clean dataset that has reliable data for full time managers by state. Let’s see if there is a trend of job postings.

In the notebook, type:

SELECT count(*), date_added, state
FROM jobs 
WHERE job_type = ‘Full Time’
AND manager = ‘y’
GROUP BY state, date_added
ORDER BY state, date_added

It will be challenging to find a clear pattern by day, let’s group by month. Use the TRUNC function to set each date to the first of the month.

In the notebook, type:

SELECT count(*), trunc(date_added, ‘MM’), state
FROM jobs 
WHERE job_type = ‘Full Time’
AND manager = ‘y’
GROUP BY trunc(date_added, ‘MM’), state
ORDER BY state, trunc(date_added, ‘MM’)

Congratulations! You’ve cleaned up a super messy dataset and focused the data on answering a specific question. You’ve learned all sorts of queries and functions that you can use on any dataset.
Dig Deeper

Want to reference back to these queries and functions? Here’s a cheat sheet with all the queries. Happy cleaning!





Appendix: Tables

Jobs: https://www.kaggle.com/PromptCloudHQ/us-jobs-on-monstercom
country
country_code
Date_added
has_expired
job_board
job_description
job_title
job_type
location
Organization

Zipcodes: https://simplemaps.com/data/us-zips
zip
city
State_id
Appendix: Cheat Sheet

TBD after finalize the lesson. Will have all queries and functions written out as a cheat code.


